Step 1: download and convert dataset from CSV to Parquet

In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from dask.distributed import Client, LocalCluster
import dask

In [3]:
import time

In [4]:
print('pandas version: %s' % pd.__version__)
print('numpy version: %s' % np.__version__)
print('dask version: %s' % dask.__version__)

pandas version: 1.3.4
numpy version: 1.20.3
dask version: 2021.10.0


https://docs.databricks.com/_static/notebooks/koalas-benchmark-distributed-execution.html?_ga=2.216403934.95291449.1648935555-599276868.1645477063

In [5]:
filename = "taxi_dataset.txt"

In [6]:
with open(filename) as file:
    csv_files = [line.rstrip() for line in file]
# only choose yellow taxis
yellow = list(filter(lambda x: "yellow" in x, csv_files))

In [7]:
# make the list small for now
yellow = ['https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-11.csv']

In [8]:
yellow

['https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2013-11.csv']

In [9]:
import re
pattern = '/[^\/]*\.csv$/gm'

for csv_url in yellow:
#     get the filename only so I can sort by month / year
    csv_name = re.findall(r"[^\/]*\.csv$",csv_url)
    df = dd.read_csv(csv_url,dtype={'tolls_amount': 'float64'})

#     df = dd.read_csv(csv_url,
#                 dtype={'RatecodeID': 'float64',
#        'VendorID': 'float64',
#        'passenger_count': 'float64',
#        'payment_type': 'float64',
#        'tolls_amount': 'float64',
#        'trip_type': 'float64'
#       })
    df = df.repartition(npartitions=4)
    df.to_parquet(f'./tmp/trip_data_{csv_name}', write_index=False)


/Users/mk/opt/anaconda3/lib/python3.9/site-packages/dask/dataframe/io/csv.py:125: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pandas_read_text(
/Users/mk/opt/anaconda3/lib/python3.9/site-packages/dask/dataframe/io/csv.py:125: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pandas_read_text(
/Users/mk/opt/anaconda3/lib/python3.9/site-packages/dask/dataframe/io/csv.py:125: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  df = pandas_read_text(


Read in parquet to dask

In [10]:
ddf = dd.read_parquet(
    "./tmp/", 
    storage_options={"anon": True, 'use_ssl': True}
)


In [11]:
ddf.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,CMT,2013-11-25 15:53:33,2013-11-25 16:00:51,1,0.6,-73.978104,40.752966,1,N,-73.985756,40.762685,CRD,6.0,1.0,0.5,1.0,0.0,8.5
1,CMT,2013-11-25 15:24:41,2013-11-25 15:30:18,1,0.5,-73.982313,40.764827,1,N,-73.982129,40.758889,CRD,5.5,0.0,0.5,3.0,0.0,9.0
2,CMT,2013-11-25 09:43:42,2013-11-25 10:02:57,1,3.3,-73.982013,40.762507,1,N,-74.006854,40.719582,CRD,15.0,0.0,0.5,2.0,0.0,17.5
3,CMT,2013-11-25 06:49:58,2013-11-25 07:04:22,1,3.8,-73.976005,40.744481,1,N,-74.016063,40.717298,CRD,14.0,0.0,0.5,2.9,0.0,17.4
4,CMT,2013-11-25 10:02:12,2013-11-25 10:17:15,1,2.2,-73.952625,40.780962,1,N,-73.981630,40.777978,CRD,12.0,0.0,0.5,2.0,0.0,14.5


In [12]:
# try to do things

In [13]:
ddf = ddf[(ddf.tip_amount >= 1) & (ddf.tip_amount <= 5)]

In [14]:
ddf.persist()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
npartitions=4,,,,,,,,,,,,,,,,,,
,object,object,object,int64,float64,float64,float64,int64,object,float64,float64,object,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [15]:
# tryna test out profiler

In [16]:
import cProfile

In [24]:
%load_ext line_profiler

In [39]:
def demo_func():
    return df.fare_amount.mean().compute()

In [29]:
%lprun -f demo_func demo_func()

In [30]:
import dask
from dask.distributed import Client

In [32]:
client = Client()

In [40]:
%lprun
future = client.submit(demo_func)

In [41]:
%lprun
counts = future.result()

In [42]:
counts

14388451

In [47]:
from dask.distributed import get_task_stream

with get_task_stream(plot='save', filename="task-stream.html") as ts:
    df.fare_amount.std().compute()
client.profile("task-profile.html")

{'description': {'filename': '', 'name': '', 'line_number': 0, 'line': ''},
 'children': {},
 'count': 0,
 'identifier': 'root'}

In [48]:
history = ts.data

In [49]:
history

[{'worker': 'tcp://127.0.0.1:58244',
  'status': 'OK',
  'nbytes': 458,
  'thread': 123145666940928,
  'type': b'\x80\x04\x95\x15\x00\x00\x00\x00\x00\x00\x00\x8c\x08builtins\x94\x8c\x04dict\x94\x93\x94.',
  'typename': 'dict',
  'metadata': {},
  'startstops': ({'action': 'compute',
    'start': 1649535018.4626062,
    'stop': 1649535018.626162},),
  'key': "('moment_chunk-29e0f8725914cd4a38179b63bfaf44ed', 2)"},
 {'worker': 'tcp://127.0.0.1:58242',
  'status': 'OK',
  'nbytes': 458,
  'thread': 123145506013184,
  'type': b'\x80\x04\x95\x15\x00\x00\x00\x00\x00\x00\x00\x8c\x08builtins\x94\x8c\x04dict\x94\x93\x94.',
  'typename': 'dict',
  'metadata': {},
  'startstops': ({'action': 'compute',
    'start': 1649535018.462799,
    'stop': 1649535018.6279953},),
  'key': "('moment_chunk-29e0f8725914cd4a38179b63bfaf44ed', 0)"},
 {'worker': 'tcp://127.0.0.1:58241',
  'status': 'OK',
  'nbytes': 458,
  'thread': 123145683894272,
  'type': b'\x80\x04\x95\x15\x00\x00\x00\x00\x00\x00\x00\x8c\x08b

# benchmarks

In [22]:
%%time
# read_file_parquet
df = dd.read_parquet(
    "./tmp/", 
    storage_options={"anon": True, 'use_ssl': True}
)

CPU times: user 9.68 ms, sys: 4.88 ms, total: 14.6 ms
Wall time: 14.2 ms


In [135]:
%%time
# count
len(df)

CPU times: user 925 ms, sys: 292 ms, total: 1.22 s
Wall time: 979 ms


14388451

In [145]:
%%time
# count_index_length
len(df.index)

CPU times: user 13.5 s, sys: 10.8 s, total: 24.4 s
Wall time: 22.3 s


14388451

In [146]:
%%time
# mean
df.fare_amount.mean().compute()

CPU times: user 413 ms, sys: 369 ms, total: 782 ms
Wall time: 430 ms


12.445337592628967

In [147]:
%%time
# standard_deviation
df.fare_amount.std().compute()

CPU times: user 661 ms, sys: 233 ms, total: 894 ms
Wall time: 464 ms


10.068131478375324

In [148]:
%%time
# mean_of_sum
(df.fare_amount + df.tip_amount).mean().compute()

CPU times: user 508 ms, sys: 415 ms, total: 923 ms
Wall time: 562 ms


13.871358994793802

In [149]:
%%time
# sum_columns
(df.fare_amount + df.tip_amount).compute()

CPU times: user 543 ms, sys: 444 ms, total: 987 ms
Wall time: 651 ms


0           7.0
1           8.5
2          17.0
3          16.9
4          14.0
           ... 
3751890     6.7
3751891     4.5
3751892    12.5
3751893    10.5
3751894     5.6
Length: 14388451, dtype: float64

In [150]:
%%time
# mean_of_product
(df.fare_amount * df.tip_amount).mean().compute()

CPU times: user 538 ms, sys: 241 ms, total: 779 ms
Wall time: 430 ms


29.84923242756293

In [151]:
%%time
# product_columns
(df.fare_amount * df.tip_amount).compute()

CPU times: user 562 ms, sys: 248 ms, total: 810 ms
Wall time: 490 ms


0           6.00
1          16.50
2          30.00
3          40.60
4          24.00
           ...  
3751890     6.60
3751891     0.00
3751892    21.00
3751893     0.00
3751894     4.95
Length: 14388451, dtype: float64

In [152]:
%%time
# value_counts
df.fare_amount.value_counts().compute()

CPU times: user 424 ms, sys: 87.2 ms, total: 512 ms
Wall time: 378 ms


6.50      700756
6.00      694981
7.00      689765
5.50      669705
7.50      668865
           ...  
89.85          1
15.58          1
90.01          1
90.11          1
104.44         1
Name: fare_amount, Length: 1598, dtype: int64

In [154]:
%%time
# mean_of_complicated_arithmetic_operation
theta_1 = df.pickup_longitude
phi_1 = df.pickup_latitude
theta_2 = df.dropoff_longitude
phi_2 = df.dropoff_latitude
temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
       + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
ret = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
ret.mean().compute()

CPU times: user 3.36 s, sys: 2.12 s, total: 5.48 s
Wall time: 2.21 s


0.002517405869102926

In [155]:
%%time
# complicated_arithmetic_operation
phi_1 = df.pickup_latitude
theta_2 = df.dropoff_longitude
phi_2 = df.dropoff_latitude
temp = (np.sin((theta_2-theta_1)/2*np.pi/180)**2
       + np.cos(theta_1*np.pi/180)*np.cos(theta_2*np.pi/180) * np.sin((phi_2-phi_1)/2*np.pi/180)**2)
ret = 2 * np.arctan2(np.sqrt(temp), np.sqrt(1-temp))
ret.compute()


CPU times: user 3.3 s, sys: 1.92 s, total: 5.22 s
Wall time: 2.14 s


0          0.000142
1          0.000029
2          0.000480
3          0.000711
4          0.000506
             ...   
3751890    0.000179
3751891    0.000102
3751892    0.000521
3751893    0.000455
3751894    0.000117
Length: 14388451, dtype: float64

In [158]:
%%time
# groupby_statistics
df.groupby(by='passenger_count').agg(
  {
    'total_amount': ['mean', 'std'], 
    'tip_amount': ['mean', 'std']
  }
).compute()

CPU times: user 13.7 s, sys: 10.1 s, total: 23.8 s
Wall time: 19.2 s


total_amount            tip_amount          
                        mean        std       mean       std
passenger_count                                             
0                  22.088918  22.028152   0.829068  3.008576
1                  14.781876  12.082950   1.447349  2.256962
2                  15.561362  12.856999   1.397596  2.284023
3                  14.937834  11.867820   1.278434  2.125639
4                  14.924136  11.896498   1.169184  2.093030
5                  15.128892  11.944140   1.446715  2.237914
6                  14.880295  11.693202   1.418491  2.224546
7                  48.129167  33.764439   2.287500  4.562103
8                  32.668750  36.240200   3.162500  5.355621
9                  37.043636  32.179872   2.059091  2.740513
208                 3.300000   0.000000   0.000000  0.000000

In [159]:
%%time
# join_count
len(dd.merge(df, other, left_index=True, right_index=True))

CPU times: user 14.5 s, sys: 8.35 s, total: 22.8 s
Wall time: 19.4 s


28

In [160]:
%%time
# join_data
dd.merge(df, other, left_index=True, right_index=True).compute()

CPU times: user 14.3 s, sys: 8.92 s, total: 23.3 s
Wall time: 19.6 s


,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,...,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,fare_amount_mean,fare_amount_std,tip_amount_mean,tip_amount_std
0.0,CMT,2013-11-25 15:53:33,2013-11-25 16:00:51,1,0.60,-73.978104,40.752966,1,N,-73.985756,...,6.0,1.0,0.5,1.00,0.00,8.50,11.203125,5.383456,2.652500,1.012792
1.0,CMT,2013-11-25 15:24:41,2013-11-25 15:30:18,1,0.50,-73.982313,40.764827,1,N,-73.982129,...,5.5,0.0,0.5,3.00,0.00,9.00,14.038450,14.163248,2.413491,0.975695
2.0,CMT,2013-11-25 09:43:42,2013-11-25 10:02:57,1,3.30,-73.982013,40.762507,1,N,-74.006854,...,15.0,0.0,0.5,2.00,0.00,17.50,18.137027,20.456144,2.495965,0.930662
3.0,CMT,2013-11-25 06:49:58,2013-11-25 07:04:22,1,3.80,-73.976005,40.744481,1,N,-74.016063,...,14.0,0.0,0.5,2.90,0.00,17.40,10.707407,5.459958,2.341556,1.066937
4.0,CMT,2013-11-25 10:02:12,2013-11-25 10:17:15,1,2.20,-73.952625,40.780962,1,N,-73.981630,...,12.0,0.0,0.5,2.00,0.00,14.50,10.322034,4.551206,2.392373,0.921325
5.0,CMT,2013-11-25 15:18:07,2013-11-25 15:33:25,1,1.00,-73.992423,40.749517,1,N,-73.988160,...,10.0,0.0,0.5,2.22,0.00,12.72,10.689109,6.595592,2.418944,1.038912
6.0,CMT,2013-11-25 21:20:50,2013-11-25 21:26:22,1,1.10,-73.946371,40.775369,1,N,-73.953090,...,6.5,0.5,0.5,1.50,0.00,9.00,10.095833,5.731124,2.231333,1.032768
0.0,VTS,2013-11-15 01:56:00,2013-11-15 01:59:00,1,0.68,-74.003275,40.708272,1,None,-74.011542,...,4.5,0.5,0.5,0.00,0.00,5.50,11.203125,5.383456,2.652500,1.012792
1.0,VTS,2013-11-15 01:33:00,2013-11-15 01:40:00,1,1.97,-73.978715,40.763650,1,None,-73.981182,...,8.5,0.5,0.5,1.50,0.00,11.00,14.038450,14.163248,2.413491,0.975695
2.0,VTS,2013-11-15 01:31:00,2013-11-15 01:41:00,1,1.75,-73.986637,40.741360,1,None,-73.997575,...,9.5,0.5,0.5,2.00,0.00,12.50,18.137027,20.456144,2.495965,0.930662
